## IMPORTING LIBRARIES

In [1]:
import requests,zipfile,io
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
import numpy as np
import random
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
import wandb

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [3]:
print(device)

cuda


## DOWNLOADING AND UNZIPPING DATA

In [4]:
def download_data(url="https://drive.google.com/u/0/uc?id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw&export=download"):
    response=requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(response.content))
    z.extractall()

## METHODS FOR GETTING CHARACTERS FOR CORPUSS AND ADDING THEIR INDICES

In [5]:
def get_corpus(data):
    eng_corpus=set()
    hin_corpus=set()
    for i in range(0,len(data)):
        eng_word=data[0][i]
        hin_word=data[1][i]
        for ch in eng_word:
            eng_corpus.add(ch)
        for ch in hin_word:
            hin_corpus.add(ch)
        # End Delimiter
        eng_corpus.add('#')
        hin_corpus.add('#')
        hin_corpus.add('$')
        eng_corpus.add('$')
        # Start Delimiter
#         eng_corpus.add('^')
        hin_corpus.add('^')
    return hin_corpus,eng_corpus

In [6]:
def word2index(data):
    hin_corpus,eng_corpus=get_corpus(data)
    engchar_idx={}
    hinchar_idx={}
    idx_engchar={}
    idx_hinchar={}
    i=0
    for char in eng_corpus:
        engchar_idx[char]=i
        idx_engchar[i]=char
        i+=1
    i=0
    for char in hin_corpus:
        hinchar_idx[char]=i
        idx_hinchar[i]=char
        i+=1
    return engchar_idx,hinchar_idx,idx_engchar,idx_hinchar,len(eng_corpus),len(hin_corpus)

## DATA PREPROCESSING

In [7]:
def maxlen(data):
    maxlen_eng=0
    maxlen_hin=0
    for i in range(0,len(data)):
        eng_word=data[0][i]
        hin_word=data[1][i]
        if(len(eng_word)>maxlen_eng):
            maxlen_eng=len(eng_word)
        if(len(hin_word)>maxlen_hin):
            maxlen_hin=len(hin_word)
    return maxlen_eng,maxlen_hin

In [8]:
def pre_process(data,eng_to_idx,hin_to_idx):
    eng=[]
    hin=[]
    maxlen_eng,maxlen_hin=maxlen(data)
    
    unknown= eng_to_idx['$']
    for i in range(0,len(data)):
        sz=0
        eng_word=data[0][i]
        hin_word='^'+data[1][i]
        eng_word = eng_word.ljust(maxlen_eng+1, '#')
        hin_word = hin_word.ljust(maxlen_hin+1, '#')
        idx=[]
        for char in eng_word:
            if eng_to_idx.get(char) is not None:
                idx.append(eng_to_idx[char])
            else:
                idx.append(unknown)
        eng.append(idx)
        idx=[]
        for char in hin_word:
            if hin_to_idx.get(char) is not None:
                idx.append(hin_to_idx[char])
            else:
                idx.append(unknown)
        hin.append(idx)    
    return eng,hin

## LOADING OUR CUSTOM DATASET TO DATALOADER

In [9]:
class MyDataset(Dataset):
    def __init__(self, train_x,train_y, transform=None):
        self.train_x = train_x
        self.train_y = train_y
        self.transform = transform
        
    
    def __len__(self):
        return len(self.train_x)
    
    def __getitem__(self, idx):
        if self.transform:
            sample = self.transform(sample)
        return torch.tensor(self.train_x[idx]).to(device),torch.tensor(self.train_y[idx]).to(device)

def get_data():
    download_data()
    
    train_df=pd.read_csv("aksharantar_sampled/hin/hin_train.csv",header=None)
    test_df=pd.read_csv("aksharantar_sampled/hin/hin_test.csv",header=None)
    val_df=pd.read_csv("aksharantar_sampled/hin/hin_valid.csv",header=None)
    eng_to_idx,hin_to_idx,idx_to_eng,idx_to_hin,input_len,target_len=word2index(train_df)
    
    return train_df,test_df,val_df,eng_to_idx,hin_to_idx,idx_to_eng,idx_to_hin,input_len,target_len

## Seq2Seq MODEL

In [10]:
class EncoderGRU(nn.Module):
    def __init__(self,input_size,hidden_size,embedding_size,num_of_layers,batch_size,bi_directional,dropout_p=0.1):
        super(EncoderGRU,self).__init__()
        self.hidden_size=hidden_size
        self.batch_size=batch_size
        self.input_size=input_size
        self.embedding_size=embedding_size
        self.embedding=nn.Embedding(input_size,embedding_size)
        self.num_of_layers=num_of_layers
        self.bi_directional=bi_directional
        if(bi_directional=="Yes"):
            flag=True
        else:
            flag=False
        self.gru = nn.GRU(embedding_size,hidden_size,num_of_layers,bidirectional=flag)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self,input,hidden):
        embedded=self.embedding(input).view(-1,self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        output,hidden=self.gru(embedded,hidden)
    
        if self.bi_directional=="Yes":
            hidden=hidden.resize(2,self.num_of_layers,self.batch_size,self.hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2
            
        return output,hidden

    def initHidden(self):
        if(self.bi_directional=="Yes"):
            return torch.zeros(2*self.num_of_layers,self.batch_size,self.hidden_size,device=device)
        else:
            return torch.zeros(self.num_of_layers,self.batch_size,self.hidden_size,device=device)

class DecoderGRU(nn.Module):
    def __init__(self, output_size,hidden_size, embedding_size, decoder_layers,batch_size,dropout_p=0.1):
        super(DecoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size=embedding_size
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.gru = nn.GRU(embedding_size,hidden_size, decoder_layers,dropout = dropout_p)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
        self.batch_size=batch_size
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(-1, self.batch_size, self.embedding_size)
#         embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden)
        output = self.softmax(self.out(output))
        return output, hidden

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size,embedding_size,num_of_layers,batch_size,bi_directional,dropout_p=0.1):
        super(EncoderRNN,self).__init__()
        self.hidden_size=hidden_size
        self.batch_size=batch_size
        self.input_size=input_size
        self.embedding_size=embedding_size
        self.embedding=nn.Embedding(input_size,embedding_size)
        self.num_of_layers=num_of_layers
        self.bi_directional=bi_directional
        if(bi_directional=="Yes"):
            flag=True
        else:
            flag=False
        self.rnn = nn.RNN(embedding_size,hidden_size,num_of_layers,bidirectional=flag)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self,input,hidden):
        embedded=self.embedding(input).view(-1,self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        output,hidden=self.rnn(embedded,hidden)
    
        if self.bi_directional=="Yes":
            hidden=hidden.resize(2,self.num_of_layers,self.batch_size,self.hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2
            
        return output,hidden

    def initHidden(self):
        if(self.bi_directional=="Yes"):
            return torch.zeros(2*self.num_of_layers,self.batch_size,self.hidden_size,device=device)
        else:
            return torch.zeros(self.num_of_layers,self.batch_size,self.hidden_size,device=device)

class DecoderRNN(nn.Module):
    def __init__(self, output_size,hidden_size, embedding_size, decoder_layers,batch_size,dropout_p=0.1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size=embedding_size
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.rnn = nn.RNN(embedding_size,hidden_size, decoder_layers,dropout = dropout_p)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
        self.batch_size=batch_size
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(-1, self.batch_size, self.embedding_size)
#         embedded = self.dropout(embedded)
        output, hidden = self.rnn(embedded, hidden)
        output = self.softmax(self.out(output))
        return output, hidden

In [12]:
class EncoderLSTM(nn.Module):
    def __init__(self,input_size,hidden_size,embedding_size,num_of_layers,batch_size,bi_directional,dropout_p=0.1):
        super(EncoderLSTM,self).__init__()
        self.hidden_size=hidden_size
        self.batch_size=batch_size
        self.input_size=input_size
        self.embedding_size=embedding_size
        self.embedding=nn.Embedding(input_size,embedding_size)
        self.num_of_layers=num_of_layers
        self.bi_directional=bi_directional
        if(bi_directional=="Yes"):
            flag=True
        else:
            flag=False
        self.lstm = nn.LSTM(embedding_size,hidden_size,num_of_layers,bidirectional=flag)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self,input,hidden,state):
        embedded=self.embedding(input).view(-1,self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        output,(hidden,state)=self.lstm(embedded,(hidden,state))
    
        if self.bi_directional=="Yes":
            hidden=hidden.resize(2,self.num_of_layers,self.batch_size,self.hidden_size)
            state=state.resize(2,self.num_of_layers,self.batch_size,self.hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2
            state=torch.add(state[0],hidden[1])/2
            
        return output,hidden,state

    def initHidden(self):
        if(self.bi_directional=="Yes"):
            return torch.zeros(2*self.num_of_layers,self.batch_size,self.hidden_size,device=device)
        else:
            return torch.zeros(self.num_of_layers,self.batch_size,self.hidden_size,device=device)
    
    def initState(self):
        if(self.bi_directional=="Yes"):
            return torch.zeros(2*self.num_of_layers,self.batch_size,self.hidden_size,device=device)
        else:
            return torch.zeros(self.num_of_layers,self.batch_size,self.hidden_size,device=device)

class DecoderLSTM(nn.Module):
    def __init__(self, output_size,hidden_size, embedding_size, decoder_layers,batch_size,dropout_p=0.1):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size=embedding_size
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size,hidden_size,decoder_layers,dropout = dropout_p)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
        self.batch_size=batch_size
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input,hidden,state):
        embedded = self.embedding(input).view(-1, self.batch_size, self.embedding_size)
#         embedded = self.dropout(embedded)
        output,(hidden,state)=self.lstm(embedded,(hidden,state))
        output = self.softmax(self.out(output))
        return output,hidden,state

## ATTENTION MECHANISM

In [13]:
class AttnDecoder(nn.Module):
    def __init__(self,output_size,hidden_size,embedding_size,decoder_layers,batch_size,cell_type,dropout_p=0.1):
        super(AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.batch_size=batch_size
        self.cell_type=cell_type
        self.embedding_size=embedding_size
        self.decoder_layers=decoder_layers
        
        self.embedding = nn.Embedding(self.output_size, self.embedding_size)
        self.dropout = nn.Dropout(self.dropout_p)

        self.U=nn.Linear(self.hidden_size,self.hidden_size,bias=False).to(device)
        self.W=nn.Linear(self.hidden_size,self.hidden_size,bias=False).to(device)
        self.V=nn.Linear(self.hidden_size,1,bias=False).to(device)
        
        self.linear=nn.Linear(self.hidden_size,output_size,bias=True)
        self.softmax=nn.LogSoftmax()
        self.softmax1=nn.LogSoftmax(dim=2)
        if(cell_type=="GRU"):
            self.gru = nn.GRU(self.embedding_size+self.hidden_size, self.hidden_size,self.decoder_layers,dropout = dropout_p)
        if(cell_type=="LSTM"):
            self.lstm = nn.LSTM(self.embedding_size+self.hidden_size, self.hidden_size,self.decoder_layers,dropout = dropout_p)
        if(cell_type=="RNN"):
            self.rnn = nn.RNN(self.embedding_size+self.hidden_size, self.hidden_size,self.decoder_layers,dropout = dropout_p)

    def forward(self, input, hidden,encoder_outputs,word_length,state=None):
        embedded = self.embedding(input).view(-1,self.batch_size, self.embedding_size)
#         embedded = self.dropout(embedded)
        T=word_length
        temp=self.W(hidden[-1])
        c=torch.zeros(self.batch_size,self.hidden_size).to(device)
        
        
        for j in range(0,T):
            e_j=self.V(torch.tanh(self.U(encoder_outputs[j])+temp))
            alpha_j=self.softmax(e_j)
            c+=alpha_j*encoder_outputs[j]
        

        final_input=torch.cat((embedded,c.unsqueeze(0)),dim=2)
        
        
        if(self.cell_type=="GRU"):
            output,hidden=self.gru(final_input,hidden)
        if(self.cell_type=="RNN"):
            output,hidden=self.rnn(final_input,hidden)
        if(self.cell_type=="LSTM"):
            output, (hidden,state) =self.lstm(final_input,(hidden,state))
        
        output1=self.softmax1(self.linear(output))
        
        if(self.cell_type=="GRU" or self.cell_type=="RNN"):
            return output1, hidden, c
        if(self.cell_type=="LSTM"):
            return output1, hidden, state, c

In [14]:
def train(train_data,encoder,decoder,loss_fun,encoder_optimizer,decoder_optimizer,encoder_layers,decoder_layers,batch_size,hidden_size,bi_directional,cell_type,attention):
    total_loss=0
    teacher_forcing_ratio=0.5
    for i,(train_x,train_y) in enumerate(train_data):
        loss=0
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        train_x=train_x.T
        train_y=train_y.T
        timesteps=len(train_x)
        
        if cell_type=='GRU' or cell_type=='RNN':
            
            encoder_hidden=encoder.initHidden()
            encoder_output,encoder_hidden=encoder(train_x,encoder_hidden)
            if(decoder_layers>encoder_layers):
                i = decoder_layers
                decoder_hidden=encoder_hidden

                while True:
                    if(i==encoder_layers):
                        break
                    # Concatenate the two tensors along the first dimension
                    decoder_hidden = torch.cat([decoder_hidden, encoder_hidden[-1].unsqueeze(0)], dim=0)
                    i-=1

            elif(decoder_layers<encoder_layers):
                decoder_hidden=encoder_hidden[-decoder_layers:]

            else:
                decoder_hidden=encoder_hidden
        
            decoder_input = train_y[0]
            
            if(bi_directional=="Yes"):
                split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
                encoder_output=torch.add(split_tensor[0],split_tensor[1])/2
            
            
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            if use_teacher_forcing:
                for i in range(0,len(train_y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(train_x))
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input = train_y[i] 
                    else:
                        decoder_output, decoder_hidden= decoder(decoder_input, decoder_hidden)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input = train_y[i]  # Teacher forcing
            else:
                for i in range(0,len(train_y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(train_x))
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input=index
                    else:
                        decoder_output,decoder_hidden=decoder(decoder_input,decoder_hidden)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input=index
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            total_loss+=loss
        
        if cell_type=='LSTM':
    
            encoder_hidden=encoder.initHidden()
            encoder_state=encoder.initState()
            
            encoder_output,encoder_hidden,encoder_state=encoder(train_x,encoder_hidden,encoder_state)
        
            if(decoder_layers>encoder_layers):
                i = decoder_layers
                decoder_hidden=encoder_hidden
                decoder_state=encoder_state
                while True:
                    if(i==encoder_layers):
                        break
                    # Concatenate the two tensors along the first dimension
                    decoder_hidden = torch.cat([decoder_hidden, encoder_hidden[-1].unsqueeze(0)], dim=0)
                    decoder_state = torch.cat([decoder_state, encoder_state[-1].unsqueeze(0)], dim=0)
                    i-=1

            elif(decoder_layers<encoder_layers):
                decoder_hidden=encoder_hidden[-decoder_layers:]
                decoder_state=encoder_state[-decoder_layers:]

            else:
                decoder_hidden=encoder_hidden
                decoder_state=encoder_state
            
            
            if(bi_directional=="Yes"):
                split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
                encoder_output=torch.add(split_tensor[0],split_tensor[1])/2
            
            decoder_input = train_y[0]
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            if use_teacher_forcing:
                for i in range(0,len(train_y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, decoder_state, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(train_x),decoder_state)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input= train_y[i]
                    else:
                        decoder_output, decoder_hidden,decoder_state= decoder(decoder_input, decoder_hidden,decoder_state)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input = train_y[i]  # Teacher forcing
            else:
                for i in range(0,len(train_y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, decoder_state, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(train_x),decoder_state)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input=index
                    else:
                        decoder_output, decoder_hidden,decoder_state= decoder(decoder_input, decoder_hidden,decoder_state)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input=index
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            total_loss+=loss

        
        
    return total_loss.item()/len(train_y),encoder,decoder

In [15]:
def train_iter(input_data,val_data,val_y,input_len,target_len,epochs,batch_size,embedding_size,encoder_layers,decoder_layers,hidden_size,cell_type,bi_directional,dropout,beam_size,attention):
    lr=0.001
    if(cell_type=='GRU'):
        encoder=EncoderGRU(input_len,hidden_size,embedding_size,encoder_layers,batch_size,bi_directional,dropout).to(device)
        if(attention=="Yes"):
            decoder=AttnDecoder(target_len,hidden_size,embedding_size,decoder_layers,batch_size,cell_type,dropout).to(device)
        else:
            decoder=DecoderGRU(target_len,hidden_size,embedding_size,decoder_layers,batch_size,dropout).to(device)
        
    if(cell_type=='RNN'):
        encoder=EncoderRNN(input_len,hidden_size,embedding_size,encoder_layers,batch_size,bi_directional,dropout).to(device)
        if(attention=="Yes"):
            decoder=AttnDecoder(target_len,hidden_size,embedding_size,decoder_layers,batch_size,cell_type,dropout).to(device)
        else:
            decoder=DecoderRNN(target_len,hidden_size,embedding_size,decoder_layers,batch_size,dropout).to(device)
    
    if cell_type=='LSTM':
        encoder=EncoderLSTM(input_len,hidden_size,embedding_size,encoder_layers,batch_size,bi_directional,dropout).to(device)
        if(attention=="Yes"):
            decoder=AttnDecoder(target_len,hidden_size,embedding_size,decoder_layers,batch_size,cell_type,dropout).to(device)
        else:
            decoder=DecoderLSTM(target_len,hidden_size,embedding_size,decoder_layers,batch_size,dropout).to(device)

    encoder_optimizer=optim.Adam(encoder.parameters(),lr)
    decoder_optimizer=optim.Adam(decoder.parameters(),lr)
    loss_fun=nn.CrossEntropyLoss(reduction="sum")
    epoch_train_loss=[]
    epoch_val_loss=[]
    epoch_val_acc=[]
    for i in range(0,epochs):
        loss,encoder,decoder=train(input_data,encoder,decoder,loss_fun,encoder_optimizer,decoder_optimizer,
                                   encoder_layers,decoder_layers,batch_size,hidden_size,bi_directional,
                                   cell_type,attention)
        val_predictions,val_loss=eval(val_data,encoder,decoder,encoder_layers,decoder_layers,
                                  batch_size,hidden_size,bi_directional,cell_type,attention)
        
        epoch_val_loss.append(val_loss)
        epoch_train_loss.append(loss/51200)
        
        val_acc=accuracy(val_predictions,val_y)
        epoch_val_acc.append(val_acc)
        print(loss/51200,val_loss,val_acc)
    
#     train_predictions,t=eval(input_data,encoder,decoder,encoder_layers,decoder_layers,batch_size,hidden_size,bi_directional,cell_type,attention)
    return epoch_train_loss,epoch_val_loss,epoch_val_acc,encoder,decoder,encoder_layers,decoder_layers

In [16]:
def eval(input_data,encoder,decoder,encoder_layers,decoder_layers,batch_size,hidden_size,bi_directional,cell_type,attention):
    with torch.no_grad():
        loss_fun=nn.CrossEntropyLoss(reduction="sum")
        total_loss=0
        pred_words=list()
        for x,y in input_data:
            loss=0
            decoder_words=[]
            x=x.T
            y=y.T
            encoder_hidden=encoder.initHidden()
            timesteps=len(x)
            if cell_type=='GRU' or cell_type=='RNN':

                encoder_hidden=encoder.initHidden()
                encoder_output,encoder_hidden=encoder(x,encoder_hidden)
                if(decoder_layers>encoder_layers):
                    i = decoder_layers
                    decoder_hidden=encoder_hidden

                    while True:
                        if(i==encoder_layers):
                            break
                        # Concatenate the two tensors along the first dimension
                        decoder_hidden = torch.cat([decoder_hidden, encoder_hidden[-1].unsqueeze(0)], dim=0)
                        i-=1

                elif(decoder_layers<encoder_layers):
                    decoder_hidden=encoder_hidden[-decoder_layers:]

                else:
                    decoder_hidden=encoder_hidden

                decoder_input = y[0]

                if(bi_directional=="Yes"):
                    split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
                    encoder_output=torch.add(split_tensor[0],split_tensor[1])/2

                for i in range(0,len(y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(x))
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), y[i])
                        index=index.squeeze()
                        decoder_input=index
                        decoder_words.append(index.tolist())
                    else:
                        decoder_output,decoder_hidden=decoder(decoder_input,decoder_hidden)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), y[i])
                        index=index.squeeze()
                        decoder_input=index
                        decoder_words.append(index.tolist())
                decoder_words=np.array(decoder_words)
                pred_words.append(decoder_words.T)
                total_loss+=loss.item()


            if cell_type=='LSTM':

                encoder_hidden=encoder.initHidden()
                encoder_state=encoder.initState()

                encoder_output,encoder_hidden,encoder_state=encoder(x,encoder_hidden,encoder_state)

                if(decoder_layers>encoder_layers):
                    i = decoder_layers
                    decoder_hidden=encoder_hidden
                    decoder_state=encoder_state
                    while True:
                        if(i==encoder_layers):
                            break
                        # Concatenate the two tensors along the first dimension
                        decoder_hidden = torch.cat([decoder_hidden, encoder_hidden[-1].unsqueeze(0)], dim=0)
                        decoder_state = torch.cat([decoder_state, encoder_state[-1].unsqueeze(0)], dim=0)
                        i-=1

                elif(decoder_layers<encoder_layers):
                    decoder_hidden=encoder_hidden[-decoder_layers:]
                    decoder_state=encoder_state[-decoder_layers:]

                else:
                    decoder_hidden=encoder_hidden
                    decoder_state=encoder_state


                if(bi_directional=="Yes"):
                    split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
                    encoder_output=torch.add(split_tensor[0],split_tensor[1])/2
                decoder_input = y[0]

                for i in range(0,len(y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, decoder_state, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(x),decoder_state)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), y[i])
                        index=index.squeeze()
                        decoder_input=index
                        decoder_words.append(index.tolist())
                    else:
                        decoder_output, decoder_hidden,decoder_state= decoder(decoder_input, decoder_hidden,decoder_state)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), y[i])
                        index=index.squeeze()
                        decoder_input=index
                        decoder_words.append(index.tolist())
                decoder_words=np.array(decoder_words)
                pred_words.append(decoder_words.T)
                total_loss+=loss.item()


    predictions=[]
    for batch in pred_words:
        for word in batch:
            predictions.append(word)

    return predictions,total_loss/(len(predictions)*len(predictions[0]))

In [17]:
def accuracy(predictions,y):
    count=0
#     print(len(y),len(y[0]))
    for i in range(0,len(predictions)):
        p=predictions[i]
        if np.array_equal(p,y[i]):
            count+=1
    return (count/len(predictions))*100

## MODEL

In [18]:
# epochs=20
# batchsize=64
# embedding_size=256
# encoder_layers=2
# decoder_layers=3
# hidden_size=256
# cell_type="LSTM"
# bi_directional="Yes"
# dropout=0.2
# beam_size=16
# attention="No"

# train_df,test_df,val_df,eng_to_idx,hin_to_idx,idx_to_eng,idx_to_hin,input_len,target_len=get_data()

# train_x,train_y = pre_process(train_df,eng_to_idx,hin_to_idx)
# test_x,test_y = pre_process(test_df,eng_to_idx,hin_to_idx)
# val_x,val_y = pre_process(val_df,eng_to_idx,hin_to_idx)

# train_dataset=MyDataset(train_x,train_y)
# test_dataset=MyDataset(test_x,test_y)
# val_dataset=MyDataset(val_x,val_y)


# train_dataloader=DataLoader(train_dataset,batch_size=batchsize)
# test_dataloader=DataLoader(test_dataset,batch_size=batchsize)
# val_dataloader=DataLoader(val_dataset,batch_size=batchsize)

In [19]:
# epoch_train_loss,epoch_val_loss,epoch_val_acc,encoder,decoder,encoder_layers,decoder_layers=train_iter(train_dataloader,val_dataloader,val_y,input_len,target_len,epochs,batchsize,embedding_size,encoder_layers,decoder_layers,hidden_size,cell_type,bi_directional,dropout,beam_size,attention)

# print(epoch_val_acc)

# test_predictions,loss=eval(test_dataloader,encoder,decoder,encoder_layers,decoder_layers,batchsize,hidden_size,bi_directional,cell_type,attention)

# test_accuracy=accuracy(test_predictions,test_y)

# print(test_accuracy)

# train_predictions,loss=eval(train_dataloader,encoder,decoder,encoder_layers,decoder_layers,batchsize,hidden_size,bi_directional,cell_type,attention)
# train_accuracy=accuracy(train_predictions,train_y)

# print(train_accuracy)

## INTEGRATING WITH WANDB

In [20]:
def wandb_run_sweeps(train_dataset,val_dataset,test_dataset,train_y,val_y,test_y,input_len,target_len):
    
    config = {
        "project":"CS6910_Assignment3",
        "method": 'bayes',
        "metric": {
        'name': 'acc',
        'goal': 'maximize'
        },
        'parameters' :{
        "epochs": {"values":[15,20,25]},
        "batchsize": {"values": [64,128,256]},
        "embedding_size": {"values":[256, 512,1024]},
        "hidden_size": {"values":[256, 512,1024]},
        "encoder_layers": {"values":[2,3,4]},
        "decoder_layers": {"values":[2,3,4]},
        "cell_type": {"values":["RNN","GRU","LSTM"]},
        "bi_directional":{"values":["Yes"]},
        "dropout":{"values":[0.1,0.2,0.5]},
        "attention":{"values":["No"]},
        "beam_size":{"values":[5,10,15]}
        }
    }
    def train_rnn():
        wandb.init()

        name='_CT_'+str(wandb.config.cell_type)+"_BS_"+str(wandb.config.batchsize)+"_EPOCH_"+str(wandb.config.epochs)+"_ES_"+str(wandb.config.embedding_size)+"_HS_"+str(wandb.config.hidden_size)
        
        
        train_dataloader=DataLoader(train_dataset,batch_size=wandb.config.batchsize)
        test_dataloader=DataLoader(test_dataset,batch_size=wandb.config.batchsize)
        val_dataloader=DataLoader(val_dataset,batch_size=wandb.config.batchsize)
        
        epoch_train_loss,epoch_val_loss,epoch_val_acc,encoder,decoder,encoder_layers,decoder_layers=train_iter(train_dataloader,val_dataloader,val_y,input_len,target_len,wandb.config.epochs,wandb.config.batchsize,wandb.config.embedding_size,wandb.config.encoder_layers,wandb.config.decoder_layers,wandb.config.hidden_size,wandb.config.cell_type,wandb.config.bi_directional,wandb.config.dropout,wandb.config.beam_size,wandb.config.attention)

        for i in range(wandb.config.epochs):
            wandb.log({"loss":epoch_train_loss[i]})
            wandb.log({"val_loss":epoch_val_loss[i]})
            wandb.log({"val_acc":epoch_val_acc[i]})
            wandb.log({"epoch": (i+1)})
        wandb.log({"validation_accuracy":epoch_val_acc[-1]})    
        
        train_predictions,_=eval(train_dataloader,encoder,decoder,wandb.config.encoder_layers,
                              wandb.config.decoder_layers,wandb.config.batchsize,wandb.config.hidden_size,
                              wandb.config.bi_directional,wandb.config.cell_type,wandb.config.attention)

        train_accuracy=accuracy(train_predictions,train_y)
        wandb.log({"train_accuracy":train_accuracy})
        
        test_predictions,_=eval(test_dataloader,encoder,decoder,wandb.config.encoder_layers,
                              wandb.config.decoder_layers,wandb.config.batchsize,wandb.config.hidden_size,
                              wandb.config.bi_directional,wandb.config.cell_type,wandb.config.attention)

        test_accuracy=accuracy(test_predictions,test_y)
        wandb.log({"test_accuracy":test_accuracy})
        wandb.log({"acc":epoch_val_acc[-1]})
        wandb.run.name = name
        wandb.run.save()
        wandb.run.finish()
    wandb.login(key="aecb4b665a37b40204530b0627a42274aeddd3e1")
    sweep_id=wandb.sweep(config,project="CS6910_Assignment3")
    wandb.agent(sweep_id,function=train_rnn,count=20)

In [21]:
train_df,test_df,val_df,eng_to_idx,hin_to_idx,idx_to_eng,idx_to_hin,input_len,target_len=get_data()

train_x,train_y = pre_process(train_df,eng_to_idx,hin_to_idx)
test_x,test_y = pre_process(test_df,eng_to_idx,hin_to_idx)
val_x,val_y = pre_process(val_df,eng_to_idx,hin_to_idx)

train_dataset=MyDataset(train_x,train_y)
test_dataset=MyDataset(test_x,test_y)
val_dataset=MyDataset(val_x,val_y)

wandb_run_sweeps(train_dataset,val_dataset,test_dataset,train_y,val_y,test_y,input_len,target_len)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: lj2769k5
Sweep URL: https://wandb.ai/cs22m078/CS6910_Assignment3/sweeps/lj2769k5


wandb: Agent Starting Run: z2cl8mx2 with config:
wandb: 	attention: No
wandb: 	batchsize: 64
wandb: 	beam_size: 15
wandb: 	bi_directional: Yes
wandb: 	cell_type: LSTM
wandb: 	decoder_layers: 4
wandb: 	dropout: 0.1
wandb: 	embedding_size: 1024
wandb: 	encoder_layers: 3
wandb: 	epochs: 15
wandb: 	hidden_size: 256
wandb: Currently logged in as: cs22m078. Use `wandb login --relogin` to force relogin


1.0143575613839286 0.5913486743257159 6.25
0.5714248511904763 0.4972603235925947 16.845703125
0.4634253801618303 0.4400637366232418 20.7763671875
0.41369410923549105 0.4103298829424949 23.73046875
0.37575198218936007 0.4005688322441919 26.8310546875
0.3472204299200149 0.39597362953992116 29.833984375
0.317063482375372 0.37313248749290195 27.9052734375
0.29717261904761905 0.380551444987456 30.6640625
0.28629350934709824 0.37137774874766666 31.1279296875
0.26753502255394346 0.3863221662385123 31.005859375
0.2546694800967262 0.3807802359972681 32.51953125
0.23892120361328126 0.39524541830732707 31.8359375
0.22990315755208332 0.4001039192080498 31.0546875
0.21178862072172622 0.40624770273764926 31.689453125
0.20801170712425596 0.3986086366432054 31.6650390625


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


...

0.7633150809151786 0.4447657607850574 20.0439453125
0.4011928013392857 0.37048236316158656 25.87890625
0.32726643880208334 0.3652804210072472 32.763671875
0.2668686930338542 0.3564740518728892 35.8154296875
0.24630382719494048 0.34880085573309944 34.765625
0.21974654424758186 0.34608026487486704 36.083984375
0.18987439836774553 0.3639016463643029 37.5732421875
0.16021873837425596 0.3668081306275867 36.3037109375
0.13746679396856398 0.3899397225607009 34.8876953125
0.11451663789295015 0.4164829956633704 36.3525390625
0.10123794555664062 0.425079508906319 36.376953125
0.0849784923735119 0.43019638175056096 35.7421875
0.07641787574404763 0.44197555099214825 36.0595703125
0.06512513660249257 0.46050052486714865 36.2548828125
0.058810828072684146 0.49097793442862375 36.8408203125
0.05873615809849331 0.4888986476830074 36.03515625
0.04715756734212239 0.5085118554887318 36.9384765625
0.04335879734584263 0.5227621566681635 36.71875
0.04320492699032738 0.5298183574562981 36.62109375
0.039465361

acc,▁
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▅▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁
train_accuracy,▁
val_acc,▁▃▆▇▇▇█▇▇██▇▇▇█▇█████▇▇▇█
val_loss,▄▂▂▁▁▁▂▂▂▃▄▄▄▅▆▆▆▇▇██▇▇██
validation_accuracy,▁
acc,36.40137
epoch,25
loss,0.03319


wandb: Agent Starting Run: 9kmp2z62 with config:
wandb: 	attention: No
wandb: 	batchsize: 256
wandb: 	beam_size: 5
wandb: 	bi_directional: Yes
wandb: 	cell_type: LSTM
wandb: 	decoder_layers: 2
wandb: 	dropout: 0.1
wandb: 	embedding_size: 1024
wandb: 	encoder_layers: 3
wandb: 	epochs: 25
wandb: 	hidden_size: 1024


1.0006211635044644 0.5689415704636347 5.224609375
0.49442452566964284 0.43135699204036165 22.3388671875
0.3992920212518601 0.40981892460868474 28.515625
0.3538254801432291 0.3744643117700304 27.7099609375
0.3223595028831845 0.35934424967992873 31.396484375
0.29057774135044645 0.3806578673067547 32.9345703125
0.26714088076636905 0.3555788681620643 33.740234375
0.23276476178850447 0.3650965591271718 34.0087890625
0.21910228910900298 0.3574165049053374 33.4716796875
0.1896871802920387 0.3818764090538025 33.154296875
0.1745669265020461 0.39612490222567603 34.4970703125
0.16145746140252976 0.39242874014945256 33.1298828125
0.146232183547247 0.40888698895772296 33.935546875
0.13149332682291667 0.4257855103129432 34.423828125
0.12009142194475446 0.42903208023025874 34.6923828125
0.11856634957449776 0.44025933742523193 32.861328125
0.1112275623139881 0.44058296510151457 33.544921875
0.10410496303013392 0.4471499579293387 33.3251953125
0.08839385259719122 0.473510898294903 33.30078125
0.0774868

acc,▁
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁
train_accuracy,▁
val_acc,▁▅▇▆▇████████████████████
val_loss,█▃▃▂▁▂▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▅▆▆
validation_accuracy,▁
acc,33.93555
epoch,25
loss,0.06391
